# **1. Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **2. Change Working directory, list file contents**

In [ ]:
import os
#os.chdir("/content/drive/Shareddrives/KernzaPhenoMethods/StemQuantification") # change this to the path of your project folder

!pwd
!ls

/content/drive/Shareddrives/KernzaPhenoMethods/StemQuantification
Documents      IlastikOutput  RawImages		 TrainingImages  YOLOOutput
IlastikModels  kernza_stems   test_inference.py  YOLOModels


# **3. Install Ultralytics, do checks, check for gpu**

If an error message _"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running."_ is returned, Inference will done on CPU which is extremely **SLOW**.

---
Please check your colab runtime environment:

In [ ]:
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import ultralytics
ultralytics.checks()
!nvidia-smi

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


Tue Jun 13 22:20:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **4. Import dependencies, set parameters**

In [ ]:
from ultralytics import YOLO
import glob
import os
import numpy as np
import pandas as pd
import torch
torch.cuda.empty_cache()
import time

import gc
gc.collect()

import cv2
from readQR import ReadQR
import readQR.wechat_artefacts as artefacts
art_dir = artefacts.__path__[0]

img_sz = 2048


# set the inference parameters
name = 'yolov8_model_'+str(img_sz)+'_1_test_salina' # set the name to give the results folder
project = 'YOLO_results'
save = True # save image results  ######################################################################
save_txt = False # save results to *.txt
save_conf = False # save confidences in --save-txt labels
show_labels = False   # hide labels
show_conf = False # hide confidences
line_width = 2
batch = -1 # batch size
visualize = False # visualize model features
conf_thres = 0.28 # confidence threshold
iou_thres = 0.55 # NMS IoU threshold
imgsz = img_sz # inference size (pixels)
exist_ok = True # if True, it overwrites current 'name' saving folder #####################################
half = True # use FP16 half-precision inference True/False
cache = True # use cache images for faster inference

img_fmt = '.JPG' # define the image format

image_folder = "Garett_Object_Detection/Test_salina" # insert path to folder with images here
base_folder = os.path.basename(image_folder)

img_paths = glob.glob(os.path.join(image_folder, '**/*'+img_fmt), recursive=True) #get the path of each image in folder and subfolders
# exclude file paths wit "B_"
img_paths = [x for x in img_paths if "B_" not in x]

# shuffle the list of images
np.random.shuffle(img_paths)

num_imgs = len(img_paths)
print("NUmber of images found = "+str(num_imgs))

nx = 10 # number of images to process at a time
img_paths2 = [img_paths[i:i + nx] for i in range(0, len(img_paths), nx)]

# load the model
model_location = 'Garett_Object_Detection/kernza_stems/yolov8_model_'+str(img_sz)+'_1/weights/best.pt' # change this to the location of your model
model = YOLO(model_location)

NUmber of images found = 842


# **5. Do Inference on the image folder, save the stem counts in a csv file**

In [ ]:
# initialize QR reader
qr_reader = ReadQR(artefact_path=art_dir)
show = None # None, 'single', 'continuous' # preview results 'single' or 'continuous' (video)

for i, img_paths in enumerate(img_paths2):
  save_name = 'stem_count_'+base_folder+'_'+str(i)+'.csv' # change name when you change folder

  # read QR code if able
  Qr_Names = []
  for img_path in img_paths:
    img = cv2.imread(img_path, cv2.COLOR_BGR2RGB)
    results = qr_reader.decode(img, show=show)
    if len(results) == 0:
      print('No QR code detected for "'+img_path+'"')
      Qr_Names.append("QR code not detected!!!")
    else:
      Qr_Names.append(results[0])
  
  # do inference and save
  tic = time.time()
  img_results = model.predict(source=img_paths, save=save, save_txt=save_txt, save_conf=save_conf, show_labels=show_labels, show_conf=show_conf,
                              line_width=line_width, visualize=visualize, project=project, name=name, imgsz=imgsz, conf=conf_thres, half=half,
                              iou=iou_thres, exist_ok=exist_ok, batch=batch, cache=cache
  )

  elapsed = 1000*(time.time()-tic)
  time_per_image = elapsed/num_imgs
  print("Infernce time per image = "+str(time_per_image)+ " ms")

  # loop through the results, get stem count, save
  # Predictions = pd.DataFrame(columns = ['ImagePath','StemCount'])
  Predictions = pd.DataFrame(columns = ['ImagePath','QRcodeInfo','StemCount'])
  n = 0
  for img_result in img_results:
    # preds = img_result.__getitem__(0)
    preds = img_result
    try:
      preds = preds.cpu().numpy()
    except:
      print('Prediction was on CPU')
    
    n_dets = int(len(preds))
    im_name = img_paths[n]
    qr_info = Qr_Names[n]

    Predictions.at[n,'ImagePath'] = im_name
    Predictions.at[n,'StemCount'] = n_dets
    Predictions.at[n,'QRcodeInfo'] = qr_info
    n+=1

  # save results in csv
  Predictions.to_csv(save_name)